In [1]:
import http.client
import urllib.parse
import pandas as pd
import json

API_ENDPOINT = "api.open-meteo.com"
PATH = "/v1/forecast"
LAT = "57.682089684462454"
LONG = "11.983288826091862"

params = {
    "latitude": LAT,
    "longitude": LONG,
    "hourly": "temperature_2m,relativehumidity_2m,windspeed_10m,precipitation,visibility,snowfall"
}
params_str = urllib.parse.urlencode(params)
url = f"{PATH}?{params_str}"

conn = http.client.HTTPSConnection(API_ENDPOINT)
conn.request("GET", url)
response = conn.getresponse()

if response.status == 200:
    response_data = response.read()
    weather_data = json.loads(response_data)
    hourly_data = weather_data["hourly"]
    data_dict = {
        'Temperature': [hourly_data['temperature_2m'][0]],
        'Humidity': [hourly_data['relativehumidity_2m'][0]],
        'Wind_Speed': [hourly_data['windspeed_10m'][0]],
        'Precipitation': [hourly_data['precipitation'][0]],
        'Visibility': [hourly_data['visibility'][0]],
        'Snowfall': [hourly_data['snowfall'][0]]
    }
    weather_df = pd.DataFrame(data_dict)
else:
    print("Failed to retrieve weather data.")

weather_df


,Temperature,Humidity,Wind_Speed,Precipitation,Visibility,Snowfall
0,4.5,80,11.9,0.0,24140.0,0.0


In [3]:
import os
from bike_data_scraper.s3_client.s3_handler import S3Handler

# Setup
WEATHER_KEY = os.environ["WEATHER_KEY"]
BIKES_KEY = os.environ["BIKES_KEY"]
SOURCE_BUCKET = os.environ["S3_SOURCE_BUCKET"]
s3_handler = S3Handler()

def lambda_handler(event, context):
    try:
        weather_data, bikes_data = fetch_data_from_s3()
        # Now weather_data and bikes_data are Pandas dataframes
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        raise e

def fetch_data_from_s3():
    weather_data = s3_handler.get_data_from_s3(SOURCE_BUCKET, WEATHER_KEY)
    bikes_data = s3_handler.get_data_from_s3(SOURCE_BUCKET, BIKES_KEY)

    if weather_data is None or bikes_data is None:
        raise ValueError("Missing data for weather or bikes")

    return weather_data, bikes_data



ModuleNotFoundError: No module named 'bike_data_scraper'

In [5]:
from datetime import timedelta, datetime

import boto3
from boto3.dynamodb.conditions import Attr


datetime.now() + timedelta(days=1)



datetime.datetime(2023, 11, 2, 17, 43, 18, 74759)

In [ ]:
import os
import boto3
from datetime import datetime
from io import StringIO
import boto3
import pandas as pd
import json



SOURCE_BUCKET = "danneftw-dscrap-bucket"
WEATHER_KEY = "weather_data.csv"
BIKES_KEY = "output3.csv"

def get_data_from_s3(bucket_name: str, key: str) -> pd.DataFrame:
    s3_client = boto3.client("s3", region_name="eu-north-1")  # Need to declare s3_client here
    obj = s3_client.get_object(Bucket=bucket_name, Key=key)
    df = pd.read_csv(obj["Body"])
    if df.empty:
        raise Exception("No data found in S3 bucket")
    return df

weather_data = get_data_from_s3(SOURCE_BUCKET, WEATHER_KEY)
bikes_data = get_data_from_s3(SOURCE_BUCKET, BIKES_KEY)

weather_data.head()

In [7]:
import pandas as pd
import boto3

def read_csv_from_s3(bucket_name: str, file_path: str) -> pd.DataFrame:
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket_name, Key=file_path)
    df = pd.read_csv(obj['Body'])
    return df

bucket_name = "danneftw-dscrap-bucket"
file_path = "processed/two_weeks/station_bikes/2023-08-09-2023-11-02/StationaryStations.csv"
dataframe = read_csv_from_s3(bucket_name, file_path)

dataframe.columns

Index(['AvailableBikes', 'StationId', 'Distance', 'stationId', 'IsOpen',
       'timestamp', 'BikeIds', 'LastUpdate', 'Long', 'Lat', 'Name', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'Temperature', 'Humidity',
       'Windspeed', 'Precipitation', 'Visibility', 'Snowfall',
       'TotalAvailableBikes', 'IsWeekend'],
      dtype='object')

In [85]:
import pandas as pd
import boto3

def read_csv_from_s3(bucket_name: str, file_path: str) -> pd.DataFrame:
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket_name, Key=file_path)
    df = pd.read_csv(obj['Body'])
    return df

bucket_name = "danneftw-dscrap-bucket"
file_path = "processed/station_bikes/2023-08-09-2023-11-02/StationaryStations.csv"
dataframe = read_csv_from_s3(bucket_name, file_path)


In [88]:
dataframe

,AvailableBikes,StationId,Distance,stationId,IsOpen,timestamp,BikeIds,LastUpdate,Long,Lat,...,Hour,Minute,Temperature,Humidity,Windspeed,Precipitation,Visibility,Snowfall,TotalAvailableBikes,IsWeekend
0,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:10:09.361000+00:00,['710976'],/Date(1694427009437+0200)/,11.953708,57.700368,...,10,10,27.5,45,2.1,0.0,24140.0,0.0,1,0
1,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:20:09.277000+00:00,['710976'],/Date(1694427609379+0200)/,11.953708,57.700368,...,10,20,27.5,45,2.1,0.0,24140.0,0.0,1,0
2,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:30:09.159000+00:00,['710976'],/Date(1694428209242+0200)/,11.953708,57.700368,...,10,30,27.5,45,2.1,0.0,24140.0,0.0,1,0
3,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:40:09.365000+00:00,['710976'],/Date(1694428809480+0200)/,11.953708,57.700368,...,10,40,27.5,45,2.1,0.0,24140.0,0.0,1,0
4,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:50:09.586000+00:00,['710976'],/Date(1694429409702+0200)/,11.953708,57.700368,...,10,50,27.5,45,2.1,0.0,24140.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726664,4,30921953,1173,Engelbrektsgatan,True,2023-11-02 03:50:09.148000+00:00,"['711999', '710881', '710280', '710121']",/Date(1698897009227+0100)/,11.976702,57.698429,...,3,50,9.8,83,8.2,0.0,24140.0,0.0,4,0
1726665,4,30921953,1173,Engelbrektsgatan,True,2023-11-02 04:00:09.331000+00:00,"['711999', '710881', '710280', '710121']",/Date(1698897609433+0100)/,11.976702,57.698429,...,4,0,9.1,86,8.2,0.0,24140.0,0.0,4,0
1726666,4,30921953,1173,Engelbrektsgatan,True,2023-11-02 04:10:09.351000+00:00,"['711999', '710881', '710280', '710121']",/Date(1698898209421+0100)/,11.976702,57.698429,...,4,10,9.1,86,8.2,0.0,24140.0,0.0,4,0
1726667,4,30921953,1173,Engelbrektsgatan,True,2023-11-02 04:20:09.432000+00:00,"['711999', '710881', '710280', '710121']",/Date(1698898809533+0100)/,11.976702,57.698429,...,4,20,9.1,86,8.2,0.0,24140.0,0.0,4,0


In [95]:
dataframe

,AvailableBikes,StationId,Distance,stationId,IsOpen,timestamp,BikeIds,LastUpdate,Long,Lat,...,Hour,Minute,Temperature,Humidity,Windspeed,Precipitation,Visibility,Snowfall,TotalAvailableBikes,IsWeekend
0,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:10:09.361000+00:00,['710976'],/Date(1694427009437+0200)/,11.953708,57.700368,...,10,10,27.5,45,2.1,0.0,24140.0,0.0,1,0
1,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:20:09.277000+00:00,['710976'],/Date(1694427609379+0200)/,11.953708,57.700368,...,10,20,27.5,45,2.1,0.0,24140.0,0.0,1,0
2,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:30:09.159000+00:00,['710976'],/Date(1694428209242+0200)/,11.953708,57.700368,...,10,30,27.5,45,2.1,0.0,24140.0,0.0,1,0
3,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:40:09.365000+00:00,['710976'],/Date(1694428809480+0200)/,11.953708,57.700368,...,10,40,27.5,45,2.1,0.0,24140.0,0.0,1,0
4,1,270950259,1567,Norra Allégatan,True,2023-09-11 10:50:09.586000+00:00,['710976'],/Date(1694429409702+0200)/,11.953708,57.700368,...,10,50,27.5,45,2.1,0.0,24140.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726664,4,30921953,1173,Engelbrektsgatan,True,2023-11-02 03:50:09.148000+00:00,"['711999', '710881', '710280', '710121']",/Date(1698897009227+0100)/,11.976702,57.698429,...,3,50,9.8,83,8.2,0.0,24140.0,0.0,4,0
1726665,4,30921953,1173,Engelbrektsgatan,True,2023-11-02 04:00:09.331000+00:00,"['711999', '710881', '710280', '710121']",/Date(1698897609433+0100)/,11.976702,57.698429,...,4,0,9.1,86,8.2,0.0,24140.0,0.0,4,0
1726666,4,30921953,1173,Engelbrektsgatan,True,2023-11-02 04:10:09.351000+00:00,"['711999', '710881', '710280', '710121']",/Date(1698898209421+0100)/,11.976702,57.698429,...,4,10,9.1,86,8.2,0.0,24140.0,0.0,4,0
1726667,4,30921953,1173,Engelbrektsgatan,True,2023-11-02 04:20:09.432000+00:00,"['711999', '710881', '710280', '710121']",/Date(1698898809533+0100)/,11.976702,57.698429,...,4,20,9.1,86,8.2,0.0,24140.0,0.0,4,0


In [99]:
missing_visibility_df = dataframe[dataframe['Visibility'].isnull()]

# Get the timestamps where 'Visibility' is missing
missing_visibility_timestamps = missing_visibility_df['timestamp']

# Display the results
print("Dates where 'Visibility' is missing:")
print(missing_visibility_timestamps)

Dates where 'Visibility' is missing:
3119       2023-08-29 13:00:09.169000+00:00
3120       2023-08-29 13:10:09.226000+00:00
3121       2023-08-29 13:20:09.201000+00:00
3122       2023-08-29 13:30:09.165000+00:00
3123       2023-08-29 13:40:08.915000+00:00
                         ...               
1717684    2023-08-31 17:10:09.038000+00:00
1717685    2023-08-31 17:20:09.674000+00:00
1717686    2023-08-31 17:30:09.396000+00:00
1717687    2023-08-31 17:40:09.374000+00:00
1717688    2023-08-31 17:50:09.270000+00:00
Name: timestamp, Length: 46908, dtype: object


In [116]:
df = dataframe
df['Visibility'].unique()

array([24140., 21980., 18200., 15400., 17100.,  6860., 23060., 18260.,
        7040.,  5740., 14480., 16800., 10500.,  4280., 12120.,  9620.,
        9100., 17640., 18500., 15660., 22840., 17060., 21940., 19740.,
       17540., 17140., 16740., 16340., 18940., 21540., 22780., 21400.,
       20040., 23980., 23840., 23680., 23140., 22140., 21140., 20980.,
       20800., 20640., 17320., 14020., 10700., 12060., 13400., 14760.,
       17880., 21020., 23420., 22700.,    nan, 12940., 14180., 14240.,
       17760.,  9940., 18420.,  7360., 22280., 19080., 17480., 12380.,
       21000., 23380.,  4300., 11400., 14200., 22720., 18660., 11980.,
       19620., 18900., 19100.,  9900., 23800., 18340., 16700., 21840.,
        7260., 14000., 13180.,  6960., 21960., 15960.,  5200., 22440.,
       15880., 16900., 15080., 18740., 15360., 13360.,  9920., 19820.,
        9220., 21620., 16760., 11300., 24040., 23640., 18880., 13140.,
       11960., 20620., 13560., 20260., 14560., 20100., 11280., 23240.,
      

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

COLUMNS_TO_KEEP = [
    "IsOpen",
    "Long",
    "Lat",
    "Year",
    "Month",
    "Day",
    "Hour",
    'Minute',
    "Temperature",
    "Humidity",
    "Windspeed",
    "Precipitation",
    "Visibility",
    "Snowfall",
    "IsWeekend",
    "TotalAvailableBikes",
]

df = dataframe[COLUMNS_TO_KEEP]

X = df.drop("TotalAvailableBikes", axis=1)
y = df["TotalAvailableBikes"]

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()

model.fit(xtrain, ytrain)

model.score(xtest, ytest)


ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [79]:
df_mossen = dataframe[dataframe['stationId']== "Polstjärnegatan"]

print(df_mossen["TotalAvailableBikes"].min())
print(df_mossen["TotalAvailableBikes"].max())
print(df_mossen["TotalAvailableBikes"].mean())
print(df_mossen["Long"].unique())
print(df_mossen["Lat"].unique())

2
16
5.649949341438703
[11.933084]
[57.710429]


In [45]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import boto3
import os

# Define the S3 bucket and folder path
bucket_name = 'sagemaker-eu-north-1-796717305864'
folder_path = 'sagemaker/sklearncontainer/train-test-two-weeks/'

# Create an S3 client
s3_client = boto3.client('s3')

# Function to download the dataset from S3
def download_from_s3(filename):
    s3_client.download_file(bucket_name, f'{folder_path}{filename}', filename)

# Downloading the data from S3
download_from_s3('xtest2.csv')
download_from_s3('xtrain2.csv')
download_from_s3('ytest2.csv')
download_from_s3('ytrain2.csv')

# Load the data
x_train = pd.read_csv('xtrain2.csv')
y_train = pd.read_csv('ytrain2.csv')
x_test = pd.read_csv('xtest2.csv')
y_test = pd.read_csv('ytest2.csv')

# Initialize the RandomForestRegressor with default parameters
rf = RandomForestRegressor()

# Train the model
rf.fit(x_train, y_train.values.ravel()) # y_train.values.ravel() to convert it to 1D array if y_train is a DataFrame

# Make predictions
predictions = rf.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Display predictions
prediction_df = pd.DataFrame(data={'Actual': y_test.iloc[:,0], 'Predicted': predictions})
print(prediction_df.head())

# Clean up the files to not clutter your notebook's directory
os.remove('xtrain2.csv')
os.remove('xtest2.csv')
os.remove('ytrain2.csv')
os.remove('ytest2.csv')


Mean Squared Error: 0.5563026118860795
   Actual  Predicted
0      13      11.53
1      17      17.00
2      10      10.00
3       6       5.98
4      11      11.03


In [80]:
import http.client
import urllib.parse
from predictor_handler import PredictorHandler
import json

API_ENDPOINT = "api.open-meteo.com"
PATH = "/v1/forecast"


def fetch_weather_data(lat, lon):
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m,relativehumidity_2m,windspeed_10m,precipitation,visibility,snowfall",
    }
    params_str = urllib.parse.urlencode(params)
    url = f"{PATH}?{params_str}"

    conn = http.client.HTTPSConnection(API_ENDPOINT)
    conn.request("GET", url)
    response = conn.getresponse()

    if response.status == 200:
        response_data = response.read()
        weather_data = json.loads(response_data)
        hourly_data = weather_data["hourly"]
        return {
            "Temperature": hourly_data["temperature_2m"][0],
            "Humidity": hourly_data["relativehumidity_2m"][0],
            "Wind_Speed": hourly_data["windspeed_10m"][0],
            "Precipitation": hourly_data["precipitation"][0],
            "Visibility": hourly_data["visibility"][0],
            "Snowfall": hourly_data["snowfall"][0],
        }
    else:
        raise Exception("Failed to retrieve weather data.")





LAT = 57.710429
LONG = 11.933084

weather_data = fetch_weather_data(LAT, LONG)

# Prepare the input DataFrame for prediction
input_df = PredictorHandler(
    IsOpen=True,
    Long=LONG,
    Lat=LAT,
    Year=2023,
    Month=11,
    Day=3,
    Hour=9,
    Minute=40,
    Temperature=weather_data["Temperature"],
    Humidity=weather_data["Humidity"],
    Wind_Speed=weather_data["Wind_Speed"],
    Precipitation=weather_data["Precipitation"],
    Visibility=weather_data["Visibility"],
    Snowfall=weather_data["Snowfall"],
    IsWeekend=0,
).create_dataframe()

# Assuming you have already deployed your SKLearn model to SageMaker and created an endpoint named 'random-forest-endpoint-1'
result = rf.predict(input_df)

# Output the result
print(f"Number of bikes at location is {result}")
input_df

Number of bikes at location is [12.92]


,IsOpen,Long,Lat,Year,Month,Day,Hour,Minute,Temperature,Humidity,Wind_Speed,Precipitation,Visibility,Snowfall,IsWeekend
0,True,11.933084,57.710429,2023,11,3,9,40,10.1,89,20.5,0.5,17000.0,0.0,0


In [83]:
df_mossen = dataframe[dataframe['stationId']== "Polstjärnegatan"]

print(df_mossen["TotalAvailableBikes"].min())
print(df_mossen["TotalAvailableBikes"].max())
print(df_mossen["TotalAvailableBikes"].mean())
print(df_mossen["Long"].unique())
print(df_mossen["Lat"].unique())

2
16
5.649949341438703
[11.933084]
[57.710429]


In [84]:
dataframe

,AvailableBikes,StationId,Distance,stationId,IsOpen,timestamp,BikeIds,LastUpdate,Long,Lat,...,Hour,Minute,Temperature,Humidity,Windspeed,Precipitation,Visibility,Snowfall,TotalAvailableBikes,IsWeekend
0,5,28045585,2484,Linnéplatsen (A),True,2023-10-18 00:00:18.009125,"['711963', '711856', '711664', '711204', '7111...",/Date(1697587218103+0200)/,11.951773,57.690247,...,0,0,2.0,96,3.3,0.0,24140.0,0.0,5,0
1,5,28045585,2484,Linnéplatsen (A),True,2023-10-18 00:10:18.581522,"['711963', '711856', '711664', '711204', '7111...",/Date(1697587818685+0200)/,11.951773,57.690247,...,0,10,2.0,96,3.3,0.0,24140.0,0.0,5,0
2,5,28045585,2484,Linnéplatsen (A),True,2023-10-18 00:20:18.436062,"['711963', '711856', '711664', '711204', '7111...",/Date(1697588418531+0200)/,11.951773,57.690247,...,0,20,2.0,96,3.3,0.0,24140.0,0.0,5,0
3,5,28045585,2484,Linnéplatsen (A),True,2023-10-18 00:30:18.143687,"['711963', '711856', '711664', '711204', '7111...",/Date(1697589018254+0200)/,11.951773,57.690247,...,0,30,2.0,96,3.3,0.0,24140.0,0.0,5,0
4,5,28045585,2484,Linnéplatsen (A),True,2023-10-18 00:40:17.817040,"['711963', '711856', '711664', '711204', '7111...",/Date(1697589617929+0200)/,11.951773,57.690247,...,0,40,2.0,96,3.3,0.0,24140.0,0.0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275274,7,30921953,1173,Engelbrektsgatan,True,2023-10-31 23:10:18.758052,"['711999', '711252', '710881', '710841', '7102...",/Date(1698793818901+0100)/,11.976702,57.698429,...,23,10,6.8,74,10.1,0.0,24140.0,0.0,7,0
275275,7,30921953,1173,Engelbrektsgatan,True,2023-10-31 23:20:18.796382,"['711999', '711252', '710881', '710841', '7102...",/Date(1698794418904+0100)/,11.976702,57.698429,...,23,20,6.8,74,10.1,0.0,24140.0,0.0,7,0
275276,7,30921953,1173,Engelbrektsgatan,True,2023-10-31 23:30:18.581008,"['711999', '711252', '710881', '710841', '7102...",/Date(1698795018688+0100)/,11.976702,57.698429,...,23,30,6.8,74,10.1,0.0,24140.0,0.0,7,0
275277,7,30921953,1173,Engelbrektsgatan,True,2023-10-31 23:40:19.024793,"['711999', '711252', '710881', '710841', '7102...",/Date(1698795619129+0100)/,11.976702,57.698429,...,23,40,6.8,74,10.1,0.0,24140.0,0.0,7,0
